#### Artificial Intelligence for Robotics
# Lesson 1: Localization

## Intro

In this course you will learn how to program self-driving cars! Specifically, in this unit, you will
learn how to program a localizer, which is a directional aid used to assist machines in making
informed navigation decisions.
Over the past decade computer scientists have developed methods using sensors, radars and
software to program vehicles with the ability to sense their own location, the locations of
other vehicles and navigate a charted course.
Stanford University Professor and director of the Stanford Artificial Intelligence Lab,
Sebastian Thrun's autonomous cars, Stanley and Junior illustrate the progress that has been
made in this field over the last decade. Additionally, the Google Driverless Car Project seeks to
further research and development to make driverless cars a viable option for people
everywhere. Driverless cars sound neat, huh? Want to program your own?

Let's get started!


## The Problem of Localization
Localization is the ability for a machine to locate itself in space. Consider a robot lost in space. Within its environment, how can the robot locate where it is? Rather than install a GPS device in our robot, we are going to write a program to implement localization. Our localization program will reduce the margin of error considerably compared to a GPS device, whose margin of error can be as high as ten meters. For our robot to successfully and accurately navigate itself through space, we are looking for a margin of error between two and ten centimeters.

Imagine a robot resides in a one-dimensional world, so somewhere along a straight line, with no idea where it is in this world. For an example of such a world, we can imagine a long, narrow hallway where it is only possible to move forward or backwards; sideways motion is impossible. Since our robot is completely clueless about its location, it believes that every point in this one dimensional world is equally likely to be its current position. We can describe this mathematically by saying that the robot's probability function is uniform (the same) over the sample space (in this case, the robot's one-dimensional world).
If we were to draw a graph of this probability function with probability on the vertical axis and location on the horizontal axis, we would draw a straight, level line. This line describes a uniform probability function, and it represents the state of maximum confusion.



<img src = "./Img1.png"/>

Assume there are three landmarks, which are three doors that all look alike and we can distinguish a door from a non-door area.If the robot senses it is next to a door, how does this affect our belief — or the probability that the robot is near a door?

<img src = "./Img2.png">

In the new function, there are three bumps aligned with the location of the doors. Since the robot has just sensed that it is near a door, it assigns these locations greater probability (indicated by the bumps in the graph) whereas, all of the other locations have decreased belief.

This function represents another probability distribution, called the posterior belief where the function is defined after the robot's sense measurement has been taken. Copyright © 2014 Udacity, Inc. All Rights Reserved. The posterior function is the best representation of the robot's current belief, where each bump represents the robot's evaluation of its position relative to a door. However, the possibility of making a bad measurement constantly looms over robotics, and over the course of this class we will see various ways to handle this problem.


In [34]:
p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    #
    n = len(p)
    q=range(len(p))
    for i in range(n):
        q[i] = p[(i-U)%len(p)]*pExact + p[(i-U+1)%len(p)]*pUndershoot + p[(i-U-1)%len(p)]*pOvershoot        
    return q

print move(p, 1)

[0.0, 0.0, 0.8, 0.0, 0.0]


New Change